# Libraries import

In [20]:
import itertools
import os
import sys
from tqdm import tqdm

import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from boltons.iterutils import pairwise
from geopy.distance import vincenty
from osgeo import gdal
from ipykernel import kernelapp as app

%matplotlib notebook
import matplotlib.pyplot as plt

from atra.utils import *
from atra.transport_flow_and_failure_functions import *

# Data folder overview

In [21]:
data_folder = "/Users/lena/Dropbox/Hackathon/rv"

In [22]:
#INPUT FILE
administrative_path = os.path.join(
    data_folder, 'data','admin', 'stluc_administrative.shp')

administrative = gpd.read_file(administrative_path)

edge_shapefile = os.path.join(
    data_folder, 'data','roads', 'stluc_roads_footpaths_all.shp')


edge_id_column= 'index'

hazard_shapefile = os.path.join(
        data_folder, 'data', 'hazards', '1m_sea-level.shp')

In [23]:
edge = gpd.read_file(edge_shapefile)
edge

,ROADS,ID,LABLE,geometry
0,Roads,0,0,LINESTRING (516311.9421530419 1546261.86762159...
1,Roads,0,0,LINESTRING (516329.2120042045 1546346.95641493...
2,Roads,0,0,LINESTRING (516155.649523778 1546628.605286161...
3,Roads,0,0,LINESTRING (516230.509739378 1546751.332925011...
4,Roads,0,0,LINESTRING (516529.0546131508 1546093.26689042...
5,Roads,0,0,LINESTRING (516156.3295237934 1546630.27525823...
6,Roads,0,0,LINESTRING (516179.5394730767 1546701.93413422...
7,Roads,0,0,LINESTRING (515891.8681398974 1546374.57152438...
8,Roads,0,0,LINESTRING (515774.4955065394 1546838.30757349...
9,Roads,0,0,LINESTRING (515887.0381489714 1546360.08175332...


In [25]:
intersections_roads_path = os.path.join(
    data_folder, 'results', 'intersections_roads_merged.csv')

roads_haz_percent_exp_path = os.path.join(
    data_folder, 'results', 'roads_hazards_%_exp.csv')

In [26]:
def line_length(line, ellipsoid='WGS-84'):
    """Length of a line in meters, given in geographic coordinates.
    Adapted from https://gis.stackexchange.com/questions/4022/looking-for-a-pythonic-way-to-calculate-the-length-of-a-wkt-linestring#answer-115285
    Args:
        line: a shapely LineString object with WGS-84 coordinates.
        ellipsoid: string name of an ellipsoid that `geopy` understands (see http://geopy.readthedocs.io/en/latest/#module-geopy.distance).
    Returns:
        Length of line in kilometers.
    """
    if line.geometryType() == 'MultiLineString':
        return sum(line_length(segment) for segment in line)

    return sum(
        vincenty(tuple(reversed(a)), tuple(reversed(b)), ellipsoid=ellipsoid).kilometers
        for a, b in pairwise(line.coords)
    )

In [27]:
def networkedge_hazard_intersection(edge_shapefile, hazard_shapefile, output_shapefile,edge_id_column):
    """Intersect network edges and hazards and write results to shapefiles
    Parameters
    ----------
    edge_shapefile
        Shapefile of network LineStrings
    hazard_shapefile
        Shapefile of hazard Polygons
    output_shapefile
        String name of edge-hazard shapefile for storing results
    Outputs
    -------
    output_shapefile
        - edge_id - String name of intersecting edge ID
        - length - Float length of intersection of edge LineString and hazard Polygon
        - geometry - Shapely LineString geometry of intersection of edge LineString and hazard Polygon
    """
    print ('* Starting {} and {} intersections'.format(edge_shapefile,hazard_shapefile))
    line_gpd = gpd.read_file(edge_shapefile)
    #line_gpd.crs = {'init': 'epsg:4326'}
    #line_gpd = line_gpd.to_crs({'init': 'epsg:4326'})
    poly_gpd = gpd.read_file(hazard_shapefile)
    #poly_gpd = poly_gpd.to_crs({'init': 'epsg:4326'})
    
    if len(line_gpd.index) > 0 and len(poly_gpd.index) > 0:
        line_gpd.columns = map(str.lower, line_gpd.columns)
        poly_gpd.columns = map(str.lower, poly_gpd.columns)

        line_bounding_box = line_gpd.total_bounds
        line_bounding_box_coord = list(itertools.product([line_bounding_box[0], line_bounding_box[2]], [
                                       line_bounding_box[1], line_bounding_box[3]]))
        line_bounding_box_geom = Polygon(line_bounding_box_coord)
        line_bounding_box_gpd = gpd.GeoDataFrame(pd.DataFrame(
            [[1], [line_bounding_box_geom]]).T, crs='epsg:4326')
        line_bounding_box_gpd.columns = ['ID', 'geometry']

        poly_bounding_box = poly_gpd.total_bounds
        poly_bounding_box_coord = list(itertools.product([poly_bounding_box[0], poly_bounding_box[2]], [
                                       poly_bounding_box[1], poly_bounding_box[3]]))
        poly_bounding_box_geom = Polygon(poly_bounding_box_coord)
        poly_bounding_box_gpd = gpd.GeoDataFrame(pd.DataFrame(
            [[1], [poly_bounding_box_geom]]).T, crs='epsg:4326')
        poly_bounding_box_gpd.columns = ['ID', 'geometry']

        poly_sindex = poly_bounding_box_gpd.sindex

        selected_polys = poly_bounding_box_gpd.iloc[list(
            poly_sindex.intersection(line_bounding_box_gpd['geometry'].iloc[0].bounds))]
        if len(selected_polys.index) > 0:
            data = []
            poly_sindex = poly_gpd.sindex
            for l_index, lines in line_gpd.iterrows():
                intersected_polys = poly_gpd.iloc[list(
                    poly_sindex.intersection(lines.geometry.bounds))]
                for p_index, poly in intersected_polys.iterrows():
                    if (lines['geometry'].intersects(poly['geometry']) is True) and (poly.geometry.is_valid is True):
                        if line_length(lines['geometry']) > 1e-3:
                            data.append({edge_id_column: lines[edge_id_column], 'length': 1000.0*line_length(lines['geometry'].intersection(
                                poly['geometry'])), 'geometry': lines['geometry'].intersection(poly['geometry'])})
                        else:
                            data.append({edge_id_column: lines[edge_id_column], 'length': 0, 'geometry': lines['geometry']})
            if data:
                intersections_data = gpd.GeoDataFrame(
                    data, columns=[edge_id_column, 'length', 'geometry'], crs='epsg:4326')
                #intersections_data = intersections_data.to_crs({'init':'epsg:4326'})
                intersections_data.to_file(output_shapefile)

    return intersections_data
    #return line_gpd, poly_gpd.to_crs({'init':'epsg:2006'})
    #del line_gpd, poly_gpd

In [28]:
def load_hazard(data_folder, hazard_id):  
    hazard_path = os.path.join(
        data_folder, 'data', 'hazards', '{}.shp'.format(hazard_id))
    hazards = geopandas.read_file(hazard_path)
    
    if hazards.crs != {'init': 'epsg:4326'}:
        hazards = hazards.to_crs({'init': 'epsg:4326'})
    return hazards

In [51]:
hazard_ids = ['1m_sea-level','4m_storm-surge','flashflooding','landslide_susceptibility']

In [29]:
hazard_ids = ['flashflooding']

In [30]:
all_intersections=[]
for hazard_id in hazard_ids:
    #OUTPUT FILE
    hazard_shapefile = os.path.join(
        data_folder, 'data', 'hazards', '{}.shp'.format(hazard_id))
    output_path = os.path.join(
        data_folder, 'results', 'intersect_road_{}.shp'.format(hazard_id))
    intersections_data = networkedge_hazard_intersection(edge_shapefile, hazard_shapefile,  output_path, edge_id_column)
    intersections_data = intersections_data.rename(columns={
        'length': hazard_id
    })
    all_intersections.append(intersections_data)
all_intersections = pd.concat(all_intersections, axis=0, sort=False)
all_intersections.to_csv(intersections_roads_path)
all_intersections.head()

* Starting /Users/lena/Dropbox/Hackathon/rv/data/roads/stluc_roads_footpaths_all.shp and /Users/lena/Dropbox/Hackathon/rv/data/hazards/flashflooding.shp intersections


AttributeError: 'NoneType' object has no attribute 'bounds'

In [12]:
#change crs of each shapefile

#for hazard.... -> intersections_data = intersections_data.to_crs({{'init':'epsg:2006'}})

,index,1m_sea-level,geometry,4m_storm-surge,flashflooding,landslide_susceptibility
0,65,5.978131,LINESTRING (-60.91110959310816 13.763594110027...,NaN,NaN,NaN
1,297,29.149873,LINESTRING (-60.92283226787809 14.091276287061...,NaN,NaN,NaN
2,327,18.311823,LINESTRING (-60.92739802939644 14.090715382163...,NaN,NaN,NaN
3,327,60.158648,LINESTRING (-60.92773060784874 14.090390844397...,NaN,NaN,NaN
4,337,173.695081,LINESTRING (-60.91214473694544 13.761988985622...,NaN,NaN,NaN
5,337,72.805370,(LINESTRING (-60.91235391890537 13.76258944131...,NaN,NaN,NaN
6,337,43.096518,LINESTRING (-60.91234434708676 13.762543252414...,NaN,NaN,NaN
7,338,22.308743,LINESTRING (-60.91135157173445 13.763575562182...,NaN,NaN,NaN
8,347,2.766942,LINESTRING (-60.91044810650474 13.766885008952...,NaN,NaN,NaN
9,354,69.402623,(LINESTRING (-60.91110959310816 13.76359411002...,NaN,NaN,NaN


In [55]:
#more elegantly: pd.DataFrame({'roads': all_intersections.sum()})
roads_exp = pd.DataFrame(all_intersections.sum(), columns=['roads'])
roads_exp = pd.DataFrame.transpose(roads_exp)
roads_exp = roads_exp.drop(['index'], axis=1)
roads_exp

,1m_sea-level,4m_storm-surge,flashflooding,landslide_susceptibility
roads,33682.231388,89873.336487,93245.600996,67555.541488


In [ ]:
#line_gpd.length
line_gpd = gpd.read_file(edge_shapefile)
line_gpd = line_gpd['length']*1000

line_gpd.sum()

In [33]:
line_gpd

,ROADS,ID,LABLE,geometry
0,Roads,0,0,LINESTRING (516311.9421530419 1546261.86762159...
1,Roads,0,0,LINESTRING (516329.2120042045 1546346.95641493...
2,Roads,0,0,LINESTRING (516155.649523778 1546628.605286161...
3,Roads,0,0,LINESTRING (516230.509739378 1546751.332925011...
4,Roads,0,0,LINESTRING (516529.0546131508 1546093.26689042...
5,Roads,0,0,LINESTRING (516156.3295237934 1546630.27525823...
6,Roads,0,0,LINESTRING (516179.5394730767 1546701.93413422...
7,Roads,0,0,LINESTRING (515891.8681398974 1546374.57152438...
8,Roads,0,0,LINESTRING (515774.4955065394 1546838.30757349...
9,Roads,0,0,LINESTRING (515887.0381489714 1546360.08175332...


In [57]:
roads_exp_per = roads_exp.copy()
for hazard_id in hazard_ids:
    roads_exp_per[hazard_id] = round((roads_exp[hazard_id]/ line_gpd.sum())*100)
    #roads_exp_per.append(roads_per)


In [58]:
roads_exp_per.index.name = 'sector'

In [59]:
roads_exp_per

,1m_sea-level,4m_storm-surge,flashflooding,landslide_susceptibility
sector,,,,
roads,1.0,3.0,4.0,3.0


In [9]:
roads_exp_per = pd.read_csv(roads_haz_percent_exp_path, index_col='sector')
roads_exp_per['capacity'] = line_gpd.sum()
roads_exp_per['unit'] = 'km'

In [10]:
roads_exp_per.to_csv(roads_haz_percent_exp_path)
#merged_intersections.to_csv(road_haz_count_path)

## Change to crs 2006

In [16]:
for hazard_id in hazard_ids:
    road = gpd.read_file(os.path.join(
        data_folder, 'results', 'intersect_road_{}.shp'.format(hazard_id)))
    road_2006 = os.path.join(
        data_folder, 'results', 'intersect_road_2006_{}.shp'.format(hazard_id))
    road = road.to_crs({'init':'epsg:2006'})
    print(road)
    road.to_file(road_2006)

     index      length                                           geometry
0       65    5.978131  LINESTRING (517775.0412301249 1521389.58746243...
1      297   29.149873  LINESTRING (516343.7466405202 1557625.68879999...
2      327   18.311823  LINESTRING (515851.1378860915 1557561.40040908...
3      327   60.158648  LINESTRING (515815.398245105 1557525.3423552, ...
4      337  173.695081  LINESTRING (517663.9372331593 1521211.55596285...
..     ...         ...                                                ...
646  14492    9.783389  LINESTRING (519521.9190936905 1527324.80124823...
647  14492   14.775476  LINESTRING (519586.3398982479 1527321.88362576...
648  14492    5.202787  LINESTRING (519611.271397472 1527328.410436097...
649  14492    5.464899  LINESTRING (519621.2418287264 1527331.49077562...
650  14492    0.480001  LINESTRING (519656.099510145 1527350.686371159...

[651 rows x 3 columns]
      index     length                                           geometry
0        65   

DriverError: /Users/lena/Dropbox/Hackathon/rv/results/intersect_road_landslide_susceptibility.shp: No such file or directory

In [22]:
for hazard_id in hazard_ids:
    road_haz = gpd.read_file(os.path.join(
        data_folder, 'results', 'intersections_road_{}.shp'.format(hazard_id)))
    print(road_haz.head())
    road_2006 = os.path.join(
        data_folder, 'results', 'intersections_road_2006_{}.shp'.format(hazard_id))
    road = gpd.read_file(os.path.join(
    data_folder, 'data','roads', 'stluc_roads_footpaths_all.shp'))
    print(road.head())
    road_new = pd.merge(road,road_haz, how='left',on ='index')
    road_new = gpd.GeoDataFrame(road_new)
    print(road_new.head())
    road_new.to_file(road_2006)
    

   index      length                                           geometry
0     65    5.978131  LINESTRING (-60.91110959310816 13.763594110027...
1    297   29.149873  LINESTRING (-60.92283226787809 14.091276287061...
2    327   18.311823  LINESTRING (-60.92739802939644 14.090715382163...
3    327   60.158648  LINESTRING (-60.92773060784874 14.090390844397...
4    337  173.695081  LINESTRING (-60.91214473694544 13.761988985622...
   index    length                                           geometry
0      0  0.113965  LINESTRING (516311.9421530419 1546261.86762159...
1      1  0.177336  LINESTRING (516329.2120042045 1546346.95641493...
2      2  0.063529  LINESTRING (516155.649523778 1546628.605286161...
3      3  0.240696  LINESTRING (516230.509739378 1546751.332925011...
4      4  0.337132  LINESTRING (516529.0546131508 1546093.26689042...
   index  length_x                                         geometry_x  \
0      0  0.113965  LINESTRING (516311.9421530419 1546261.86762159...   
1 

AttributeError: No geometry data set yet (expected in column 'geometry'.

In [57]:
road_haz = gpd.read_file(os.path.join(
        data_folder, 'results', 'intersect_road_flashflooding.shp'))
road_haz.cr

In [58]:
road_haz.crs = {'init': 'epsg:4326'}
road_haz.to_file(os.path.join(
        data_folder, 'results', 'intersect_road_flashflooding_4326.shp'))